In [1]:
#准备资料
import numpy as np
import matplotlib.pyplot as plt
x_train_data='/home/zjy/ml hw2/data/X_train'
y_train_data='/home/zjy/ml hw2/data/Y_train'
x_test_data='/home/zjy/ml hw2/data/X_test'
output_data='/home/zjy/ml hw2/data/generative_output.csv'
np.random.seed(0) #使后面生成的随机数相同

with open(x_train_data) as f1:
    next(f1) #跳过第一行，从第二行开始读，第一行是features名称
    X_train=np.array([line.strip('\n').split(',')[1:] for line in f1],dtype=float)
    #strip('\n')表示删掉每行数据中的换行符，split(',')以每行中逗号作数据分隔，每行从下标为1的数据读起，因下标为0的是id号
    
with open(y_train_data) as f2:
    next(f2)
    Y_train=np.array([line.strip('\n').split(',')[1] for line in f2],dtype=float) #只读下标1列，即表示类别0还是1的列
    
with open(x_test_data) as f3:
    next(f3)
    X_test=np.array([line.strip('\n').split(',')[1:] for line in f3],dtype=float)

def normalize(X,train=True,column_id=None,X_mean=None,X_std=None):
#该函数用于对矩阵X做正则化操作，train为true时代表处理的是training data，为false时是testing data，由于对X_test的正则化也是依靠X_test的mean和
#std，所以为false时直接使用之前算得的mean，std。column_id是索引每个要正则化的列，是一个数，若为none返回一个数，不为none表示求所有列的mean和std，
#并返回1*n的矩阵
    eps=1e-10 #防止分母为0
    if column_id==None:
        column_id=np.arange(X.shape[1])
    if train== True:
        X_mean=np.mean(X[:,column_id],0).reshape(1,-1)
        X_std=np.std(X[:, column_id],0).reshape(1,-1)
    X[:,column_id]=(X[:,column_id]-X_mean)/(X_std+eps)
    return X,X_mean,X_std

In [2]:
#对X_train,X_test进行正则化
X_train,X_mean,X_std=normalize(X_train,train=True)
X_test,_,_=normalize(X_test,train=False,column_id=None,X_mean=X_mean,X_std=X_std) #单下划线作为临时变量名

#划分两个正态分布集，X_train_0,X_train_1
X_train_0=np.array([x for x,y in zip(X_train,Y_train) if y==0])
X_train_1=np.array([x for x,y in zip(X_train,Y_train) if y==1])

#求出两对应均值矩阵
mean_0=np.mean(X_train_0,axis=0)
mean_1=np.mean(X_train_1,axis=0)

#初始化covariance matrix
data_dim=X_train.shape[1]
cov_0=np.zeros((data_dim,data_dim))
cov_1=np.zeros((data_dim,data_dim))

#计算covariance matrix
for x in X_train_0:
    cov_0+=np.dot(np.transpose([x-mean_0]),[x-mean_0])/X_train_0.shape[0]
for x in X_train_1:
    cov_1+=np.dot(np.transpose([x-mean_1]),[x-mean_1])/X_train_1.shape[0]
    
#使两个正态分布有相同的covariance matrix
cov=(cov_0*X_train_0.shape[0]+cov_1*X_train_1.shape[0])/(X_train_0.shape[0]+X_train_1.shape[0])

In [3]:
#通过svd分解求矩阵的逆
u,s,v=np.linalg.svd(cov,full_matrices=False)
inv=np.matmul(v.T*1/s,u.T)

#计算w和b
w=np.dot(inv,mean_0-mean_1)
b=(-0.5)*np.dot(mean_0,np.dot(inv,mean_0))+0.5*np.dot(mean_1,np.dot(inv,mean_1))+np.log(float(X_train_0.shape[0])/X_train_1.shape[0])

In [4]:
def f(X,w,b):
#定义通过sigmoid函数得到预测结果并取整
    z=np.matmul(X,w)+b #matmul矩阵相乘
    sigmoid=np.clip(1/(1.0+np.exp(-z)),1e-10,1-(1e-10)) #clip是将大于1的数变成1，将小于0的数变成0
    return sigmoid
def predict(X,w,b):
    return np.round(f(X,w,b)).astype(np.int) #对预测结果取整数

def accuracy(Y_pred,Y_label):
#算predict值和label值的准确率
    return 1-np.mean(np.abs(Y_pred-Y_label)) #值越接近1预测越准确

#计算training set上的accuracy
Y_train_pred=1-predict(X_train,w,b)
print('Training accuracy:{}'.format(accuracy(Y_train_pred,Y_train)))

Training accuracy:0.8645311117664406


/tmp/ipykernel_10705/980785793.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.round(f(X,w,b)).astype(np.int) #对预测结果取整数


In [5]:
#将结果写入csv文件
pred=predict(X_test,w,b)
with open(output_data,'w')as f:
    f.write('id,label\n')
    for i,label in enumerate(pred): #enumerate能同时遍历出pred的索引和元素
        f.write('{},{}\n'.format(i,label))

/tmp/ipykernel_10705/980785793.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.round(f(X,w,b)).astype(np.int) #对预测结果取整数
